##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Como migrar o treinamento de um worker com multiplas GPUs

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/mirrored_strategy"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/mirrored_strategy.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/migrate/mirrored_strategy.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/migrate/mirrored_strategy.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Este guia demonstra como migrar os workflows de um worker com múltiplas GPUs, do TensorFlow 1 para o TensorFlow 2.

Para realizar treinamento síncrono em múltiplas GPUs de uma única máquina:

- No TensorFlow 1, você usa as APIs `tf.estimator.Estimator` com `tf.distribute.MirroredStrategy`.
- No TensorFlow 2, você pode usar [Keras Model.fit](https://www.tensorflow.org/tutorials/distribute/keras) ou [um loop de treinamento personalizado](https://www.tensorflow.org/tutorials/distribute/custom_training) com `tf.distribute.MirroredStrategy`. Saiba mais no guia [Treinamento distribuído com TensorFlow](https://www.tensorflow.org/guide/distributed_training#mirroredstrategy).

## Configuração

Comece com os imports e um dataset simples para fins de demonstração:

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [[0.8], [0.9], [1.]]

## TensorFlow 1: treinamento distribuído de um único worker com tf.estimator.Estimator

Este exemplo demonstra o workflow canônico do TensorFlow 1 para treinamento multi-GPU com um único worker. Você precisa definir a estratégia de distribuição (`tf.distribute.MirroredStrategy`) através do parâmetro `config` do `tf.estimator.Estimator`:

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

def _eval_input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(1)(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  return tf1.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

strategy = tf1.distribute.MirroredStrategy()
config = tf1.estimator.RunConfig(
    train_distribute=strategy, eval_distribute=strategy)
estimator = tf1.estimator.Estimator(model_fn=_model_fn, config=config)

train_spec = tf1.estimator.TrainSpec(input_fn=_input_fn)
eval_spec = tf1.estimator.EvalSpec(input_fn=_eval_input_fn)
tf1.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

## TensorFlow 2: Treinamento de um único worker com Keras

Ao migrar para o TensorFlow 2, você pode usar as APIs Keras com `tf.distribute.MirroredStrategy`.

Se você usar as APIs `tf.keras` para construção de modelo e Keras `Model.fit` para treinamento, a principal diferença será instanciar o modelo Keras, um otimizador e métricas no contexto de `Strategy.scope`, em vez de definir uma `config` para `tf.estimator.Estimator`.

Se você precisar usar um loop de treinamento personalizado, veja o guia [Usando tf.distribute.Strategy com loops de treinamento personalizados](https://www.tensorflow.org/guide/distributed_training#using_tfdistributestrategy_with_custom_training_loops).

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
  model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
  optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer=optimizer, loss='mse')
model.fit(dataset)
model.evaluate(eval_dataset, return_dict=True)

## Próximos passos

Para saber mais sobre treinamento distribuído com `tf.distribute.MirroredStrategy` no TensorFlow 2, confira a seguinte documentação:

- O tutorial [Treinamento distribuído em uma máquina com Keras](../../tutorials/distribute/keras)
- O tutorial [Treinamento distribuído em uma máquina com um de loop de treinamento personalizado](../../tutorials/distribute/custom_training)
- O guia [Treinamento distribuído com TensorFlow](../../guide/distributed_training)
- O guia [Usando múltiplas GPUs](../../guide/gpu#using_multiple_gpus)
- O guia [Otimize o desempenho no host único multi-GPU (com o TensorFlow Profiler)](../../guide/gpu_performance_analysis#2_optimize_the_performance_on_the_multi-gpu_single_host)